In [1]:
import nltk
from tqdm.notebook import tqdm

In [2]:
f = open("tags-processed-unique.txt", "r")
lines = f.readlines()
f.close()

In [3]:
lines = lines[0].split(", ")

In [4]:
len(lines)

1042

In [5]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ragha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ragha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
from nltk.corpus import wordnet as wn

In [6]:
def get_synonyms(word):
    print(word)
    return list(map(lambda x: x.name(), wn.synsets(word)[0].lemmas()))

In [8]:
wn.synsets(lines[0].split()[0])

[Synset('bibulous.s.01')]

In [11]:
# # Example list of words
# word_list = lines[:]

# # Remove synonyms from word list
# new_word_list = []
# for word in tqdm(word_list):
#     synonyms = set()
#     for syn in wn.synsets(word):
#         for lemma in syn.lemmas():
#             synonyms.add(lemma.name())
#     if not set(word_list) & synonyms:
#         new_word_list.append(word)
#     else:
#         print(word, "is a synonym of", list(set(word_list) & synonyms)[0])

# # Print new word list without synonyms
# print(new_word_list[:10])

In [30]:
def remove_synonyms(words):
    new_word_list = []
    for word in words:
        synsets = wn.synsets(word)
        synonyms = set()
        for synset in synsets:
            for lemma in synset.lemmas():
                if lemma.name() != word:
                    synonyms.add(lemma.name())
        if not synonyms:
            new_word_list.append(word)
        if synonyms:
            new_word_list.append(word)
    return new_word_list
new_tags = remove_synonyms(lines)
len(new_tags)

1042

In [31]:
new_tags = list(set(new_tags))
len(new_tags)

1038

In [46]:
from langdetect import detect
from googletrans import Translator

translator = Translator()

def is_english(word):
    try:
        lang = translator.detect(word)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

def remove_non_english(words):
    english_words = []
    for word in tqdm(words):
        if is_english(word):
            english_words.append(word)
        else:
            try:
                translation = translator.translate(word, dest='en')
                english_words.append(translation.text)
            except:
                pass
    return english_words

In [33]:
new_tags_eng = remove_non_english(new_tags)

  0%|          | 0/1038 [00:00<?, ?it/s]

In [67]:
new_eng_tags = []
for i in new_tags:
    check = all(list(map(lambda x: False if wn.synsets(x) == [] else True, i.split())))
    if check:
        new_eng_tags.append(i)
len(new_eng_tags)

937

In [69]:
new_eng_tags = list(set(new_eng_tags))

937

In [75]:
new_tags_singwrd = list(filter(lambda x: len(x.split()) == 1, new_eng_tags))

In [20]:
new_tags_1 = remove_synonyms(new_tags_eng)

In [88]:
new_tags_final = list(map(lambda x: x.lower(), list(set(new_tags_singwrd) - set([i for i in new_tags_singwrd if '-' in i]))))

In [75]:
# write the result
f = open("tags-processed-final.txt", "w")
for tag in new_tags_final:
    f.write(tag + ",")
f.close()

In [13]:
line = list(set(line))

In [76]:
f = open("tags-processed-final.txt", "r")
lines = f.readlines()
f.close()

In [5]:
# shuffle the list
import random
line = lines[0].split(",")
random.shuffle(line)


In [77]:
new_tags_final = line

In [78]:
len(new_tags_final)

629

In [29]:
import difflib
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [57]:
def find_root(tag_list):
    # use nltk to get the root of the words in the list
    root_list = []
    for tag in tag_list:
        root_list.append(lemmatizer.lemmatize(tag, 'v'))
    return root_list
lemmatizer.lemmatize('nostalgic', 'v')

'nostalgic'

In [47]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language="english")

In [50]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
 
ps = PorterStemmer()

In [73]:
def remove_similar_within_list(tag_list):
    # use difflib to get the most similar words in the list
    res = tag_list[:]
    for i in range(len(tag_list)):
        for j in range(i+1, len(tag_list)):
            if difflib.SequenceMatcher(None, tag_list[i], tag_list[j]).ratio() > 0.8:
                if stemmer.stem(tag_list[i]) == stemmer.stem(tag_list[j]):
                    # print(tag_list[i], tag_list[j])
                    try:
                        res.remove(tag_list[j])
                    except ValueError:
                        print("{} not in list".format(tag_list[j]))

    return res
tags_new = remove_similar_within_list(line)

emotional not in list
spirituality not in list
tradition not in list


In [74]:
to_rem = ['nostalgic','india', 'celebratory','dreams','romanticism', 'reverent', 'adoration', 'enthusiasm','humor','melancholy', 'flirtation','lover','rhythm']

for i in to_rem:
    try:
        tags_new.remove(i)
    except ValueError:
        print("{} not in list".format(i))
